In [14]:
from pathlib import Path
import pyarrow.parquet as pq
import pandas as pd

### Read parquet file

In [15]:
# read parquet file
path = Path("../data/processed/merged_20260108_174125.parquet")
table = pq.read_table(path)
df = table.to_pandas()
print(df.columns)
print(df.shape)

Index(['adresse', 'categories_de_poi', 'classements_du_poi',
       'code_postal_et_commune', 'contacts_du_poi',
       'covid19_mesures_specifiques', 'createur_de_la_donnee',
       'date_de_mise_a_jour', 'description', 'latitude', 'longitude',
       'nom_du_poi', 'periodes_regroupees', 'sit_diffuseur', 'uri_id_du_poi',
       'region', 'code_postal', 'commune', 'departement', 'types_list',
       'type_principal', 'main_category', 'sub_category', 'itineraire',
       'h3_r6', 'h3_r7', 'h3_r8', 'h3_r9', 'density_commune',
       'density_commune_norm', 'diversity_commune', 'diversity_commune_norm',
       'rating', 'review_count', 'rating_norm', 'reviews_norm',
       'popularity_norm', 'centroid_lat', 'centroid_lon', 'proximity_commune',
       'proximity_commune_log', 'proximity_commune_norm', 'category_weight',
       'category_weight_norm', 'is_open_now', 'open_hours_norm', 'open_late',
       'open_weekend', 'opening_score', 'opening_score_norm',
       'final_score_raw', 'final

In [16]:
# filtrer que lesm main main category caculé dans les itinéraires
df = df_clean[df_clean["itineraire"]== True]
print(df.shape)

(208352, 52)


In [17]:
df_main_cat = df.groupby("main_category").size().reset_index(name="count")
df_main_cat["main_category"].unique()
df_main_cat = df_main_cat.sort_values(by="count", ascending=False)

df_sub_cat = df.groupby("sub_category").size().reset_index(name="count")
df_sub_cat["sub_category"].unique()
df_sub_cat = df_sub_cat.sort_values(by="count", ascending=False)

df_type_prin = df.groupby("type_principal").size().reset_index(name="count")
df_type_prin = df_type_prin.sort_values(by="count", ascending=False)

In [18]:
import plotly.express as px

#fig = px.histogram(df_main_cat, x="main_category", y="count", title="Distribution des catégories de POI")
fig = px.pie(df_main_cat, values="count", names="main_category", title="Distribution des catégories de POI")
fig.show()  


In [19]:
fig = px.histogram(df_sub_cat, x="sub_category", y="count", title="Distribution des sous catégories de POI")
fig.show()

In [20]:
fig = px.histogram(df_type_prin, x="type_principal", y="count", title="Distribution des types principaux de POI")
fig.show()

In [21]:
missing = df.isna().sum().sort_values(ascending=False)

fig = px.bar(
    missing.reset_index(),
    x=0, y="index",
    orientation="h",
    labels={"index": "Colonne", "0": "Valeurs manquantes"}
)
fig.show()

In [22]:
df["date_de_mise_a_jour"] = pd.to_datetime(df["date_de_mise_a_jour"], errors="coerce")
df["year_month"] = df["date_de_mise_a_jour"].dt.to_period("M")
freq = df.groupby("year_month").size()

fig = px.bar(
    x=freq.index.astype(str),
    y=freq.values,
    labels={"x": "Année-Mois", "y": "Nombre de POI mis à jour"},
    title="Fréquence des mises à jour par mois"
)

fig.update_layout(xaxis_tickangle=-45)
fig.show()

C:\Users\gg\AppData\Local\Temp\ipykernel_588\1504148996.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\gg\AppData\Local\Temp\ipykernel_588\1504148996.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

